### setup

In [1]:
import gzip
import json
from typing import List
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import inspect_ai
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import torch as t
import subprocess
import contextlib
import shutil
import ast
import textwrap

In [2]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
from inspect_ai.log import read_eval_log
import re

In [3]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
IMPORT_HELPER = {
    "python": [
        "import math",
        "import re",
        "import sys",
        "import copy",
        "import datetime",
        "import itertools",
        "import collections",
        "import heapq",
        "import statistics",
        "import functools",
        "import hashlib",
        "import numpy",
        "import numpy as np",
        "import string",
        "from typing import *",
        "from collections import *",
    ],
    "go"    : [
        "math",
        "strings",
        "fmt",
        "strconv",
        "time",
        "bytes",
        "regexp",
        "sort",
        "math/rand",
        "crypto/md5",
    ],
    "cpp"   : [
        "#include<stdlib.h>",
        "#include<algorithm>",
        "#include<math.h>",
        "#include<stdio.h>",
        "#include<vector>",
        "#include<string>",
        "#include<climits>",
        "#include<cstring>",
        "#include<iostream>",
    ],
}

# instruction prepended to code problem
HUMANEVAL_INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.\n
"""

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [5]:
model = get_model(
        'hf/meta-llama/Llama-3.1-8B-Instruct', 
        device = 'auto',
        torch_dtype=t.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Data Processing

In [40]:
def stream_jsonl_all(filename: str):
    results = []
    fp = gzip.open(open(filename, "rb"), "rt")
    for line in fp:
        if any(not x.isspace() for x in line):
            results.append(json.loads(line))
    fp.close()

    return results

In [41]:
python_content = stream_jsonl_all('data/python_data.gz')
cpp_content = stream_jsonl_all('data/cpp_data.gz')
go_content = stream_jsonl_all('data/go_data.gz')
java_content = stream_jsonl_all('data/java_data.gz')
js_content = stream_jsonl_all('data/js_data.gz')
content = [python_content, cpp_content, go_content, java_content, js_content]

In [42]:
generations = stream_jsonl_all('data/python_generations.gz')
generations[0]['generation']

'    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n'

In [43]:
for lang in content:
    print(lang[0].keys())
    print()

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'import', 'docstring', 'declaration', 'canonical_solution', 'test', 'test_setup', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])



### LLM output => code  
*I need to see LLM output & transform that into code*

In [39]:
# Installing Node.JS
#     `apt-get update`  
#     `apt-get install -y curl`  
#     `curl -fsSL https://deb.nodesource.com/setup_lts.x | bash -`  
#     `apt-get install -y nodejs`  
#     verify installation: `node -v` // `npm -v` // `node -e "console.log('Node.js is working')"`  


#### Running Eval

In [37]:
# FIND CODE

def find_code_new(completion: str) -> str:
    pattern_1 = re.compile(r"```javascript\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if len(matches) == 0: raise Exception('error processing model output')

    extracted_answer = matches[0]
    extracted_answer = naive_extract_function(extracted_answer)
    return str(extracted_answer)

def naive_extract_function(code: str) -> str:
    lines = code.splitlines()
    found = False
    for line in lines:
        if line.lstrip().startswith('const'):
            lines.remove(line)
            found = True
            break  

    if not found: 
        print('error extracting function body')
        return 'errormsg'
    return "\n".join(lines)

In [38]:
def get_final_js(state, completion):
    final_code = state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
    return final_code

In [72]:
@scorer(metrics=[accuracy(), stderr()])
def js_scorer() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        idx = state.sample_id
        model_completion = state.output.completion
        processed_completion = find_code_new(model_completion)
        final_code = get_final_js(state, processed_completion)

        if 'errormsg' in processed_completion:
            print(f'error in sample: {idx}')

        try:
            result = await sandbox().exec(
                cmd=["node", "-e", final_code],
                timeout=30,
            )
        except TimeoutError:
            result = ExecResult(False, 1, "", "Verification timed out.")

        success = result.success and (result.stderr == '')

        return Score(
            value=CORRECT if success else INCORRECT,
            explanation="".join(
                ["The following verification code was executed:\n\n"]
                + [final_code]
                + [f"\n\nThe submission was incorrect\n\n{result.stderr}"]
                if not result.success
                else [""]
            ),
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                'final_code': final_code,
                'idx': idx
            },
        )

    return score



In [73]:
lang = 'js'

def humaneval_record_to_sample(record):
    model_input = HUMANEVAL_INSTRUCTION + LANG_PREFIX[lang] + '\n' + record['prompt'] 
    
    return Sample(
        id=record["task_id"],
        input=model_input,
        target=record["canonical_solution"],
        metadata={
            "prompt": record["prompt"],
            "test": record["test"],
        },
    )

humaneval_dataset = hf_dataset(
    path = 'THUDM/humaneval-x',
    name = lang,
    split = 'test',
    sample_fields = humaneval_record_to_sample,
    trust = True,
)

In [76]:
samples = 164

@task
def humaneval():
    return Task(
        dataset = humaneval_dataset[:samples],
        solver = generate(),
        scorer = js_scorer(),
        sandbox = 'local',
    )

In [78]:
epochs = 1
inspect_ai.eval(humaneval(), model = model, epochs = epochs)

Output()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


#### Checking outputs

In [21]:
idx = 0

log = read_eval_log('/root/srf-project/test_humaneval-x/logs/js_4o-mini_1.eval')
model_out = log.samples[idx].messages[-1].content

print(log.samples[idx].id)
print()
print(model_out)

JavaScript/0

```javascript
const hasCloseElements = (numbers, threshold) => {
    for (let i = 0; i < numbers.length; i++) {
        for (let j = i + 1; j < numbers.length; j++) {
            if (Math.abs(numbers[i] - numbers[j]) < threshold) {
                return true;
            }
        }
    }
    return false;
};
```


In [21]:
target_id = 145
idx = 0
model_out = ''

for i, sample in enumerate(log.samples):
    id = int(sample.id.split('/')[-1])
    if id == target_id:
        idx = i
        model_out = sample.messages[-1].content
        print(i)
        print()
        print(model_out)
        break


52

```python
def order_by_points(nums):
    """
    Sorts the given list of integers in ascending order according to the sum of their digits.
    If there are several items with similar sum of their digits, order them based on their index in original list.

    Args:
        nums (list): A list of integers.

    Returns:
        list: The sorted list of integers.
    """
    return sorted(enumerate(nums), key=lambda x: (sum(int(digit) for digit in str(abs(x[1])), x[0]))
```


In [22]:
print(find_code_new(model_out))

Error extracting function body: Generator expression must be parenthesized (<unknown>, line 12)
errormsg
